In [1]:
import os   # '''provides a way to interact with the operating system, such as navigating file directories and manipulating files.'''
import cv2   #'''provides functions for reading, writing, and manipulating image data'''
import time  # '''provides functions for measuring the execution time of code.'''
from tqdm import tqdm   #'''provides a progress bar that shows the progress of a loop or operation'''
import numpy as np   #'''multi-dimensional arrays and matrix operations'''
import tensorflow as tf #'''support for building, training, and deploying deep learning models''' 
from sklearn.model_selection import train_test_split  #'''function for splitting data into training and testing sets for machine learning models'''
from tensorflow.keras.models import Sequential, load_model  # '''`Sequential`: provides a way to create a neural network model in Keras API by stacking layers sequentially. - `load_model`: provides a way to load a saved Keras model from disk.'''
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Activation, Dropout # '''Conv2D`: provides a way to add a 2-dimensional convolution layer to a Keras model. - `Dense`: provides a way to add a fully connected layer to a Keras model. - `MaxPooling2D`: provides a way to add a 2-dimensional max pooling layer to a Keras model. - `Flatten`: provides a way to flatten the output of a layer into a 1-dimensional array. - `Activation`: provides a way to specify an activation function for a layer in a Keras model. - `Dropout`: provides a way to add a dropout layer to a Keras model to prevent overfitting.'''

In [2]:
img_size = 100
datadir = r'C:\Users\ASUS\OneDrive\Desktop\assignment\Dataset'
CATEGORIES = os.listdir(datadir)
print(CATEGORIES)

['without_mask', 'with_mask']


In [3]:
# Define two empty list for contain image data
x, y = [], []
   
def PreProcess():
    for category in CATEGORIES:
        path = os.path.join(datadir, category)
        classIndex = CATEGORIES.index(category)
        print(path)
        for imgs in tqdm(os.listdir(path)):
            img_arr = cv2.imread(os.path.join(path, imgs))
            
            # resize the image
            resized_array = cv2.resize(img_arr, (img_size, img_size))
            cv2.imshow("images", resized_array)
            cv2.waitKey(1)
            resized_array = resized_array/255.0
            x.append(resized_array)
            y.append(classIndex)
            
PreProcess()
cv2.destroyAllWindows()

C:\Users\ASUS\OneDrive\Desktop\assignment\Dataset\without_mask


100%|██████████████████████████████████████████████████████████████████████████████| 2994/2994 [00:46<00:00, 64.94it/s]


C:\Users\ASUS\OneDrive\Desktop\assignment\Dataset\with_mask


100%|██████████████████████████████████████████████████████████████████████████████| 2994/2994 [00:46<00:00, 64.04it/s]


In [4]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [5]:
X_train = np.array(X_train).reshape(-1, img_size, img_size, 3)
y_train = np.array(y_train)
X_test = np.array(X_test).reshape(-1, img_size, img_size, 3)
y_test = np.array(y_test)

In [6]:
batch_size = 32
epochs = 15

In [7]:
model = Sequential()

model.add(Conv2D(64,(3, 3), input_shape=(img_size, img_size, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(16, activation='relu'))


model.add(Dense(len(CATEGORIES)))
model.add(Activation('softmax'))

# compile the model

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        1792      
                                                                 
 activation (Activation)     (None, 98, 98, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 256)       147712    
                                                                 
 activation_1 (Activation)   (None, 47, 47, 256)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 256)      0         
 2D)                                                    

In [8]:
t1 = time.time()

model.fit(X_train, y_train, batch_size = batch_size, epochs=epochs, validation_split=0.3, verbose = 1)
model.save('{}.h5'.format("model"))

t2 = time.time()
print('Time taken: ',t2-t1)

Epoch 1/15
105/105 [==============================] - 196s 2s/step - loss: 0.2436 - accuracy: 0.9099 - val_loss: 0.1206 - val_accuracy: 0.9617
Epoch 2/15
105/105 [==============================] - 209s 2s/step - loss: 0.1145 - accuracy: 0.9603 - val_loss: 0.0896 - val_accuracy: 0.9763
Epoch 3/15
105/105 [==============================] - 208s 2s/step - loss: 0.1250 - accuracy: 0.9600 - val_loss: 0.1318 - val_accuracy: 0.9701
Epoch 4/15
105/105 [==============================] - 215s 2s/step - loss: 0.0891 - accuracy: 0.9705 - val_loss: 0.1079 - val_accuracy: 0.9659
Epoch 5/15
105/105 [==============================] - 223s 2s/step - loss: 0.1079 - accuracy: 0.9600 - val_loss: 0.0929 - val_accuracy: 0.9777
Epoch 6/15
105/105 [==============================] - 208s 2s/step - loss: 0.0849 - accuracy: 0.9752 - val_loss: 0.1043 - val_accuracy: 0.9680
Epoch 7/15
105/105 [==============================] - 213s 2s/step - loss: 0.0882 - accuracy: 0.9708 - val_loss: 0.0753 - val_accuracy: 0.9784

In [9]:
validation_loss, validation_accuracy = model.evaluate(X_test, y_test)

38/38 [==============================] - 18s 471ms/step - loss: 0.0897 - accuracy: 0.9649


## import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the model
model = load_model('model.h5')

# Define mediapipe Face detector

face_detection = mp.solutions.face_detection.FaceDetection()

# Detection function



def get_detection(frame):


    height, width, channel = frame.shape

    # Convert frame BGR to RGB colorspace

    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Detect results from the frame
    
    result = face_detection.process(imgRGB)


    try:
        for count, detection in enumerate(result.detections):

            # print(detection)

        
            # Extract bounding box information 
            
            box = detection.location_data.relative_bounding_box

            x, y, w, h = int(box.xmin*width), int(box.ymin * height), int(box.width*width), int(box.height*height)
            
    # If detection is not available then pass 
    except:
        pass

    return x, y, w, h

CATEGORIES = ['no_mask','mask' ]
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break  # exit loop if no more frames are available
    img = frame.copy()
    try:
        x, y, w, h = get_detection(frame)
        
        crop_img = img[y:y+h, x:x+w]
        
        crop_img = cv2.resize(crop_img, (100, 100))
        
        crop_img = np.expand_dims(crop_img, axis=0)
        
        # get the prediction from the model.
        prediction = model.predict(crop_img)
        print(prediction).
        index = np.argmax(prediction)
        res = CATEGORIES[index]
        if index == 0:
            color = (0, 0, 255)
        else:
            color = (0, 255, 0)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, res, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,
                                 0.8, color, 2, cv2.LINE_AA)

    except:
        pass
    
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) == ord('o'):
        break
    
cap.release()
cv2.destroyAllWindows()